In [1]:
# import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import datetime as dt
import numpy as np

In [2]:
# read in our cleaned, merged data
ML_df = pd.read_csv('resources/prepped_for_ML.csv', index_col=[0])
ML_df

,Order Date,Ship Date,Sales,Quantity,Discount,Profit,Shipping Cost,Days to Ship,Returned,Country_Afghanistan,...,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables,Order Priority_Critical,Order Priority_High,Order Priority_Low,Order Priority_Medium
Order ID,,,,,,,,,,,,,,,,,,,,,
CA-2014-AB10015140-41954,2014-11-11,2014-11-13,221.98,2,0.0,62.15,40.770,2,0.0,0,...,0,0,1,0,0,0,0,1,0,0
IN-2014-JR162107-41675,2014-02-05,2014-02-07,3709.40,9,0.1,-288.77,923.630,2,0.0,0,...,0,0,0,0,0,0,1,0,0,0
IN-2014-CR127307-41929,2014-10-17,2014-10-18,5175.17,9,0.1,919.97,915.490,1,0.0,0,...,0,0,1,0,0,0,0,0,0,1
ES-2014-KM1637548-41667,2014-01-28,2014-01-30,2892.51,5,0.1,-96.54,910.160,2,0.0,0,...,0,0,1,0,0,0,0,0,0,1
SG-2014-RH9495111-41948,2014-11-05,2014-11-06,2832.96,8,0.0,311.52,903.040,1,0.0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IN-2015-KE1642066-42174,2015-06-19,2015-06-19,65.10,5,0.0,4.50,1.010,0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
US-2014-ZD21925140-41765,2014-05-06,2014-05-10,16.72,5,0.2,3.34,1.930,4,0.0,0,...,0,0,0,0,0,0,0,1,0,0
CA-2012-ZD21925140-41147,2012-08-26,2012-08-31,8.56,2,0.0,2.48,1.580,5,0.0,0,...,0,0,0,0,0,0,0,1,0,0


In [12]:
# Order & Ship Date to datetime
ML_df['Order Date'] = pd.to_datetime(ML_df['Order Date'])
ML_df['Ship Date'] = pd.to_datetime(ML_df['Ship Date'])

## Logistic Regression - Returned

In [13]:
# Set features and targets
y = ML_df['Returned']
X = ML_df.drop(columns='Returned')

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(38467, 232)

In [15]:
# import dependency & create logistic regression model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [16]:
# train model with training data set
classifier.fit(X_train, y_train)

TypeError: The DTypes <class 'numpy.dtype[float64]'> and <class 'numpy.dtype[datetime64]'> do not have a common DType. For example they cannot be stored in a single array unless the dtype is `object`.

In [17]:
# make prediction and display predicted vs actual
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

NameError: name 'y_pred' is not defined

In [ ]:
# import dependency & calculate accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

## Linear Regression - Profit

In [8]:
y = ML_df['Profit']
X = ML_df.drop(columns='Profit')
y

Order ID
CA-2014-AB10015140-41954     62.15
IN-2014-JR162107-41675     -288.77
IN-2014-CR127307-41929      919.97
ES-2014-KM1637548-41667     -96.54
SG-2014-RH9495111-41948     311.52
                             ...  
IN-2015-KE1642066-42174       4.50
US-2014-ZD21925140-41765      3.34
CA-2012-ZD21925140-41147      2.48
MX-2013-RB1979518-41322       2.40
MX-2013-MC1810093-41416       1.80
Name: Profit, Length: 51290, dtype: float64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.